In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog

# Global variables
video_path = ""
speed_scale = 30  # km/h
car_count = 0
motorcycle_count = 0
truck_count = 0
vehicles = []
tracked_vehicle_ids = set()
iou_threshold = 0.1  # Adjust the IoU threshold as needed

# Define the region of interest (ROI) in the video
roi_x, roi_y, roi_w, roi_h = 100, 300, 1000, 400

# Function to calculate Intersection over Union (IoU) between two bounding boxes
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    x_intersection = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
    y_intersection = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
    intersection_area = x_intersection * y_intersection

    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area if union_area > 0 else 0
    return iou

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to estimate speed
def estimate_speed(prev_center, current_center, fps, scale):
    distance_pixels = calculate_distance(prev_center, current_center)
    distance_feet = distance_pixels / scale
    speed_fps = distance_feet * fps
    speed_kmph = speed_fps * 0.0003048 * 3600 / 1000  # Convert speed from fps to km/h
    return speed_kmph

# Function to process each frame
def process_frame(frame, scale, net, layer_names):
    global car_count, motorcycle_count, truck_count, vehicles, tracked_vehicle_ids

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)
    
    currCarCount = 0
    currMotorCycleCount = 0
    currTruckCount = 0

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            vehicleType = ""
            if class_id == 2:  # Car
                vehicleType = "Car"
            elif class_id == 3:  # Motorcycle
                vehicleType = "Motorcycle"
            elif class_id == 5:  # Truck
                vehicleType = "Truck"
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                # Check if the center of the vehicle is within the ROI
                if roi_x <= center_x <= roi_x + roi_w and roi_y <= center_y <= roi_y + roi_h:
                    # Generate a unique identifier for each detected vehicle
                    vehicle_id = hash((center_x, center_y, w, h))

                    # Check if the vehicle is already tracked
                    if vehicle_id not in tracked_vehicle_ids:
                        # Check IoU with existing tracked vehicles
                        new_box = (x, y, w, h)
                        is_new_vehicle = True
                        
                        for vehicle in vehicles:
                            existing_box = (vehicle['x'], vehicle['y'], vehicle['w'], vehicle['h'])
                            iou = calculate_iou(new_box, existing_box)

                            if iou > iou_threshold:
                                # This bounding box is associated with an existing tracked vehicle
                                is_new_vehicle = False
                                vehicle_id = vehicle['id']
                                tracked_vehicle_ids.add(vehicle_id)

                                # Update the bounding box coordinates
                                vehicle['x'], vehicle['y'], vehicle['w'], vehicle['h'] = x, y, w, h

                                # Display bounding box on the frame
                                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                                speed = estimate_speed((existing_box[0],existing_box[1]), (x,y), 60, 0.02)
                                cv2.putText(frame, str(round(speed, 1)), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                                break

                        if is_new_vehicle:
                            # Count the vehicle
                            if class_id == 2:  # Car
                                car_count += 1
                                currCarCount += 1
                            elif class_id == 3:  # Motorcycle
                                motorcycle_count += 1
                                currMotorCycleCount += 1
                            elif class_id == 5:  # Truck
                                truck_count += 1
                                currTruckCount += 1

                            # Store vehicle information for speed estimation
                            vehicles.append({'id': vehicle_id, 'x': x, 'y': y, 'w': w, 'h': h, 'prev_center': (center_x, center_y), 'current_center': (center_x, center_y)})

                            # Add the vehicle ID to the set of tracked IDs
                            tracked_vehicle_ids.add(vehicle_id)

                            # Display bounding box on the frame
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            cv2.putText(frame, vehicleType, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    # Remove vehicles that are outside the ROI
                    vehicles = [vehicle for vehicle in vehicles if roi_x <= vehicle['current_center'][0] <= roi_x + roi_w and roi_y <= vehicle['current_center'][1] <= roi_y + roi_h]
                    cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (255, 0, 0), 2)
                    countStr = "Cars: " + str(car_count) + ", Motorcycles: " + str(motorcycle_count) + ", Trucks: " + str(truck_count)
                    cv2.putText(frame, countStr, (roi_x, roi_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Function to open video file and start processing
def open_file():
    global video_path, car_count, motorcycle_count, truck_count, vehicles, tracked_vehicle_ids
    video_path = filedialog.askopenfilename()
    if video_path:
        car_count = 0
        motorcycle_count = 0
        truck_count = 0
        vehicles = []
        tracked_vehicle_ids = set()

        cap = cv2.VideoCapture(video_path)
        scale = 8 * 20  # pixels per square foot
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Load YOLO model
        net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
        with open('coco.names', 'r') as f:
            classes = [line.strip() for line in f.readlines()]
        layer_names = net.getUnconnectedOutLayersNames()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process the frame
            process_frame(frame, scale, net, layer_names)

            # Display the frame
            cv2.imshow('Video', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# GUI setupq
root = tk.Tk()
root.title("Vehicle Counting and Speed Estimation")

# Open File Button
open_button = tk.Button(root, text="Open Video File", command=open_file)
open_button.pack(pady=10)

root.mainloop()
